In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
TEXT_CEO = """У топ-менеджера две основные функции. Первая — добиваться поставленных целей. Вторая — научить коллег добиваться поставленных целей.

***
Мы не любим ставить перед собой большие цели. Наш мозг, все наше естество противится этому. Почему? Ответ парадоксален, но точен. Потому что боимся, что они будут достигнуты! Да, да, именно поэтому.

***
Какая у человека будет цель, такая у него будет и жизнь.

***
Если у тебя нет стратегии — значит, у тебя нет бизнеса. Гендиректор, который не имеет цели хотя бы на пять лет, не может называть себя генеральным директором.

***
Учите свою компанию учиться. Новые знания способны дать бизнесу больше, чем миллионы инвестиций.

***
Люди любят, когда их хвалят. Для многих это важнее секса и денег. Потребность во внешнем признании сидит в нас так же глубоко, как волчья боязнь всего нового, что тоже неоднократно доказано психологическими экспериментами.

***
Чего бы ты ни достиг, ты всегда находишься в точке возможной погибели

***
Что бы там ни говорили про лихие 90-е, во все времена люди ценили честность и порядочность. И если придерживаться строгих правил и выполнять обещания, то риски можно свести к минимуму всегда и везде.

***
Улыбка генерального директора не элемент косметики для корпоративной культуры. Улыбка босса стоит миллионы в самом прямом смысле этого слова. Что делают сотрудники, когда руководитель приходит утром на предприятие? Смотрят: какое у шефа настроение. И не важно, карьерист ты или добросовестный трудяга. Это наш биологический инстинкт — постоянно сверять свои действия с сигналами вожака.

***
Надо приходить на работу и делиться, делиться, делиться хорошим настроением. Заряжать и заражать. Вот увидите, ваши сотрудники неизбежно передадут свое настроение клиентам. А клиенты — это наше всё.

***
Если к вам на работу придет устраиваться Билл Гейтс, хорошенько подумайте: нужен ли он вашей компании?

***
Человеку изначально даются лишь черты характера и склонность к определенного рода занятиям, которая диктуется набором генов. Все остальное — результат его упорства, воли, а также влияния людей, которые встретились на жизненном пути, и обстоятельств.

***
Ваша задача — не просто найти достойного человека, а выбрать достойного человека, наилучшим образом подходящего для конкретной задачи.

***
Все люди важны и эффективны. Бесполезных не бывает. Главное — найти подход, который поможет раскрыть их потенциал. А это наша, менеджерская задача.

***
Ищите своего покупателя и работайте только для него. Остальные придут сами.

***
Когда требования ваших покупателей грубо противоречат вашим принципам и ценностям, выбирать нужно принципы и ценности.

***
Бизнес говорит на языке цифр. А цифры — прежде всего ответственность. Поэтому цифры у нас никто не любит.

***
Генеральный директор, который не уважает решение собственника и остается в компании исключительно из паразитических соображений, жалок. И даже если его тактика сработает, даже если глупое решение, которое принял акционер, «как-нибудь само рассосется», все равно такой менеджер никогда не достигнет по-настоящему больших высот и результатов.

***
Где-то в мозгу у вас обязательно должна быть коробка передач. Надо только научиться ею пользоваться.

***
Есть только одна вещь, которая ценнее времени. Это твоя репутация. То, каким тебя запомнят после твоего ухода из компании, из профессии, из жизни. И на эту главную ценность нельзя жалеть ни времени, ни денег."""

DATA_CEO = TEXT_CEO.split(sep='\n\n***\n')
DATA_CEO

['У топ-менеджера две основные функции. Первая — добиваться поставленных целей. Вторая — научить коллег добиваться поставленных целей.',
 'Мы не любим ставить перед собой большие цели. Наш мозг, все наше естество противится этому. Почему? Ответ парадоксален, но точен. Потому что боимся, что они будут достигнуты! Да, да, именно поэтому.',
 'Какая у человека будет цель, такая у него будет и жизнь.',
 'Если у тебя нет стратегии — значит, у тебя нет бизнеса. Гендиректор, который не имеет цели хотя бы на пять лет, не может называть себя генеральным директором.',
 'Учите свою компанию учиться. Новые знания способны дать бизнесу больше, чем миллионы инвестиций.',
 'Люди любят, когда их хвалят. Для многих это важнее секса и денег. Потребность во внешнем признании сидит в нас так же глубоко, как волчья боязнь всего нового, что тоже неоднократно доказано психологическими экспериментами.',
 'Чего бы ты ни достиг, ты всегда находишься в точке возможной погибели',
 'Что бы там ни говорили про

In [12]:
import spacy

class RoleWordsIDF():
    
    def __init__(self) -> None:
        self.vect = None
    
    def fit(self, DATA, num_words = 20):
        
        nlp = spacy.load('ru_core_news_md', disable=['tagger', 'attribute_ruler', 'senter', 'parser', 'ner'])

        df = pd.DataFrame({'data':DATA})
        data = df.data.apply(nlp)

        def is_alphanum(token):
            return not token.is_punct \
                and not token.is_currency \
                and not token.is_digit \
                and not token.is_punct \
                and not token.is_oov \
                and not token.is_space \
                and not token.is_stop \
                and not token.like_num \
                and not token.pos_ == "PROPN"

        data = data.to_frame() \
            .applymap(lambda x: ' '.join(t.lemma_ for t in x if is_alphanum(t))) \
            .data
        
        self.vect = TfidfVectorizer(norm=None)

        self.vect.fit(data)
        # print(self.vect.get_feature_names_out())
        feature_names = self.vect.get_feature_names_out()
        
        max_value = self.vect.transform(data).max(axis=0).toarray().ravel()
        sorted_by_tfidf = max_value.argsort()

        print("Признаки с наименьшими значениями tfidf")
        print(feature_names[sorted_by_tfidf[:num_words]])
        print("Признаки с наибольшими значениями tfidf")
        print(feature_names[sorted_by_tfidf[-num_words:]])
        
    def get_feature_names(self):
        return self.vect.get_feature_names_out()

In [13]:
vect = CountVectorizer()

vect.fit(DATA_CEO)
print(vect.get_feature_names_out())
sorting = np.argsort(vect.transform(DATA_CEO).toarray().sum(axis=0))[::-1]
print("Most popular words CountVectoriser")
vect.get_feature_names_out()[sorting[:20]]

['90' 'акционер' 'бесполезных' 'бизнес' 'бизнеса' 'бизнесу' 'билл'
 'биологическии' 'боимся' 'больше' 'большие' 'больших' 'босса' 'боязнь'
 'будет' 'будут' 'бы' 'бывает' 'быть' 'важнее' 'важно' 'важны' 'вам' 'вас'
 'ваша' 'вашеи' 'ваши' 'вашим' 'ваших' 'везде' 'вещь' 'влияния' 'внешнем'
 'во' 'вожака' 'возможнои' 'воли' 'волчья' 'вот' 'времена' 'времени' 'все'
 'всегда' 'всего' 'встретились' 'вторая' 'выбирать' 'выбрать' 'выполнять'
 'высот' 'где' 'геи' 'гендиректор' 'генерального' 'генеральныи'
 'генеральным' 'генов' 'главное' 'главную' 'глубоко' 'глупое' 'говорили'
 'говорит' 'грубо' 'да' 'даже' 'дать' 'даются' 'две' 'деи' 'делают'
 'делиться' 'денег' 'диктуется' 'директор' 'директора' 'директором' 'для'
 'добиваться' 'добросовестныи' 'доказано' 'должна' 'достиг' 'достигнет'
 'достигнуты' 'достои' 'его' 'если' 'естество' 'есть' 'ею' 'жалеть'
 'жалок' 'же' 'жизненном' 'жизни' 'жизнь' 'задача' 'задачи' 'занятиям'
 'запомнят' 'заражать' 'заряжать' 'знания' 'значит' 'из' 'изначально'
 'и

array(['не', 'на', 'все', 'это', 'если', 'что', 'из', 'для', 'ни',
       'делиться', 'когда', 'компании', 'то', 'которыи', 'люди', 'только',
       'человека', 'ты', 'тебя', 'бы'], dtype=object)

In [14]:
role_ceo = RoleWordsIDF()
role_ceo.fit(DATA_CEO)
print(role_ceo.get_feature_names())

Признаки с наименьшими значениями tfidf
['компания' 'цель' 'достигнуть' 'любить' 'важный' 'директор' 'мозг'
 'приходить' 'миллион' 'прийти' 'менеджер' 'бизнес' 'больший' 'говорить'
 'работа' 'хороший' 'генеральный' 'жизнь' 'сотрудник' 'деньга']
Признаки с наибольшими значениями tfidf
['минимум' 'косметика' 'язык' 'называть' 'многих' 'набор' 'человек' 'ни'
 'время' 'настроение' 'задача' 'ценность' 'клиент' 'ваш' 'добиваться'
 'поставить' 'принцип' 'решение' 'делиться' 'цифра']
['акционер' 'бизнес' 'больший' 'босс' 'боязнь' 'бояться' 'важный' 'ваш'
 'вещь' 'влияние' 'внешний' 'вожак' 'волчий' 'воля' 'время' 'встретиться'
 'выбирать' 'выбрать' 'выполнять' 'высота' 'ген' 'генеральный' 'главный'
 'глубоко' 'глупый' 'говорить' 'год' 'грубо' 'даваться' 'дать' 'делать'
 'делиться' 'деньга' 'диктоваться' 'директор' 'добиваться' 'доказать'
 'достигнуть' 'естество' 'жалеть' 'жалок' 'жизненный' 'жизнь' 'задача'
 'занятие' 'запомнить' 'заражать' 'знание' 'изначально' 'инстинкт'
 'исключительно' 'ка

In [16]:
from tkinter.tix import TEXT


TEXT_ACCAUNTANT = """Основные принципы бухгалтерского учета
Бухгалтерский учет является достаточной строгой дисциплиной. Строгой в том смысле, что существуют определенные принципы ведения учета, несоблюдение которых может привести к тому, что его данные будут недостоверными, неполными, непонятными пользователям бухгалтерской информации, а ведь на ней строится управление как отдельно взятым предприятием, так и страной в целом.

Так, в процессе многовекового становления бухгалтерского учета были определены такие принципы бухгалтерского учета:

 начисления;
 непрерывности;
 понятности;
 существенности;
 значимости;
 достоверности и правдивости;
 объективности;
 осмотрительности;
 завершенности;
 сопоставимости;
 последовательности;
 единства денежного измерения;
 конфиденциальности.
Принцип начисления
Один из основополагающих принципов. Все операции в бухгалтерском учете должны отображаться в момент их совершения, а не тогда, когда произойдет ожидаемое от этой операции действие. К примеру, вы отгрузили товар покупателю. Эта операция подлежит отображению в бухгалтерском учете уже в момент отгрузки, а не тогда, когда поступит ожидаемая прибыль.

Принцип непрерывности
Бухгалтерский учет может вестись только на том предприятии, которое видит себя действующим в обозримом будущем и не собирается прекращать деятельность.

Принцип понятности
Учет должен вестись так, чтобы его данные были понятны пользователю информации. С этой целью разработан единый порядок записи операций.

Принцип существенности
Информация бухгалтерского учета представляет собой большую ценность, поэтому при ведении учета следует заботиться о том, чтобы она подавалась по существу и представляла собой интерес для пользователя.

Принцип значимости
Этот принцип дополняет предыдущий – вся информация бухгалтерского учета должна не только подаваться по существу, но быть полезной и значимой для управления, оценки и анализа финансовой, операционной, хозяйственной деятельности.

Принцип достоверности и правдивости
Учетная информация должна подаваться без предвзятости и не может содержать ошибок.

Принцип объективности
Все без исключения проходящие на предприятии операции должны найти отображение в бухгалтерском учете и подтверждаться документально.

Принцип осмотрительности
Проводить оценку в процессе бухгалтерского учета следует с осторожностью – доходы и активы не могут быть завышены, а расходы, обязательства – занижены.

Принцип завершенности
Операция, отображенная в бухгалтерском учете должна быть логически завершенной и представленной в полном объеме.

Принцип сопоставимости
Поскольку бухгалтерский учет должен быть полезным для управления и анализа деятельности, его данные должны отображаться так из периода в период, чтобы их можно было сопоставить. С этой целью и разрабатываются унифицированные отчетные формы, учетная политика на предприятии.

Принцип последовательности
Несмотря на то, что у бухгалтерской отчетности существуют установленные сроки: месяц, квартал, декада, полгода и год, она должна быть последовательной из года, в год, с начала и до конца деятельности предприятия. Таким образом, как и при применении принципа сопоставимости можно отслеживать динамику развития предприятия и принимать на основании полученной информации необходимые управленческие решения.

Принцип единства измерения
Несмотря на то, что в процессе ведения деятельности информация поступает в денежных, натуральных и трудовых измерителях, ее сопоставление и анализ может проводиться только с использованием денежных показателей. Так реализуются и другие принципы, важные для принятия решений – принципы сопоставимости и последовательности.

Принцип конфиденциальности
Данные бухгалтерского учета представляют собой коммерческую тайну. Если исключения возможны, они оговариваются с бухгалтером заранее. За нарушение этого принципа в РФ предусмотрена административная и уголовная ответственность.
бухгалтерские услуги, бухгалтерский учёт, бухгалтер Новосибирск
На указанных выше принципах бухгалтерского учета строится нормативная и законодательная база РФ в этой сфере."""

DATA_ACCAUNTANT = TEXT_ACCAUNTANT.split(sep='\n')
DATA_ACCAUNTANT

['Основные принципы бухгалтерского учета',
 'Бухгалтерский учет является достаточной строгой дисциплиной. Строгой в том смысле, что существуют определенные принципы ведения учета, несоблюдение которых может привести к тому, что его данные будут недостоверными, неполными, непонятными пользователям бухгалтерской информации, а ведь на ней строится управление как отдельно взятым предприятием, так и страной в целом.',
 '',
 'Так, в процессе многовекового становления бухгалтерского учета были определены такие принципы бухгалтерского учета:',
 '',
 ' начисления;',
 ' непрерывности;',
 ' понятности;',
 ' существенности;',
 ' значимости;',
 ' достоверности и правдивости;',
 ' объективности;',
 ' осмотрительности;',
 ' завершенности;',
 ' сопоставимости;',
 ' последовательности;',
 ' единства денежного измерения;',
 ' конфиденциальности.',
 'Принцип начисления',
 'Один из основополагающих принципов. Все операции в бухгалтерском учете должны отображаться в момент их совершения, а не тогда, когда 

In [17]:
role_accauntant = RoleWordsIDF()
role_accauntant.fit(DATA_ACCAUNTANT)

Признаки с наименьшими значениями tfidf
['информация' 'деятельность' 'сопоставимость' 'подаваться' 'анализ'
 'пользователь' 'последовательность' 'ведение' 'процесс' 'управление'
 'завершённость' 'значимость' 'измерение' 'исключение'
 'конфиденциальность' 'непрерывность' 'единство' 'объективность'
 'правдивость' 'осмотрительность']
Признаки с наибольшими значениями tfidf
['обозримый' 'найти' 'нормативный' 'несоблюдение' 'оговариваться'
 'неполный' 'недостоверный' 'натуральный' 'нарушение' 'бухгалтерский'
 'учёт' 'принцип' 'предприятие' 'представлять' 'денежный' 'момент'
 'период' 'ожидать' 'операция' 'год']
